In [1]:
#note the op will not be having the op since i have a potato pc

In [2]:
import tensorflow as tf 

tf.random.set_seed(1)

In [3]:
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step
